## Bluestream


1. https://www.gem.wiki/Blue_Stream_Gas_Pipeline
1. https://www.nsenergybusiness.com/projects/blue-stream-natural-gas-pipeline/#

In [1]:
import json
import re
import requests
from haversine import haversine
from ipyleaflet import (Map, basemaps, basemap_to_tiles, WMSLayer, Marker, GeoJSON, AwesomeIcon, ImageOverlay, 
                        ScaleControl, LayersControl, ZoomControl, FullScreenControl, DrawControl, LayerGroup)

In [ ]:
# https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html
# basemaps.Esri.WorldImagery
# basemap=basemap_to_tiles(basemaps.Esri.WorldImagery),
# url='https://www.gebco.net/data_and_products/gebco_web_services/web_map_service/mapserv',
# https://tiles.emodnet-bathymetry.eu/wmts/1.0.0/WMTSCapabilities.xml
# https://tiles.emodnet-bathymetry.eu/2020/baselayer/{TileMatrixSet}/{TileMatrix}/{TileCol}/{TileRow}.png
# https://tiles.emodnet-bathymetry.eu/2020/baselayer/web_mercator/{TileMatrix}/{TileCol}/{TileRow}.png
OSM=basemaps.OpenStreetMap.Mapnik
worldImagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
gebco =WMSLayer(
    url='https://wms.gebco.net/mapserv',
    layers='GEBCO_LATEST',
    format='image/png',
    transparent=True,
    attribution='GEBCO'
)
emodnet =WMSLayer(
    url='https://tiles.emodnet-bathymetry.eu/2020/baselayer/web_mercator/1/{TileCol}/{TileRow}.png',
    tms=True
)
m = Map(
    basemap=OSM,
    center=(42.9, 36.4),
    zoom=6,
    zoom_control=False,
    attribution_control=False
    )
m.add_control(ZoomControl(position='topright'))
m.add_control(LayersControl(position='topleft'))
m.add_control(ScaleControl(position='bottomright', metric=True, imperial=False))
m.add_control(FullScreenControl())

In [3]:
def calc_line_KPs(coords, rnd=True, cum=True):
    KP = [0]
    for ii, coord1 in enumerate(coords[:-1]):
        long1, lat1 = coord1
        long2, lat2 = coords[ii+1]
        #print(coord1, coord2)
        dist = haversine((lat1, long1), (lat2, long2))
        #distances.append(dist)
        #print(type(dist), dist, len(distances))
        if cum:
            KP.append(dist + KP[-1])
        #if len(distances)>0 and cum:
        #    dist = dist + distances[-1]
        #    KP.append(dist)
        #    #print(dist, distances[-1])
        else:
            KP.append(dist)
        #print(distances)
    if rnd:
        KP = [round(ii, 3) for ii in KP]
    return KP

In [4]:
payload = {
    "request": "getfeatureinfo",
    "service": "wms",
    "crs": "EPSG:4326",
    "layers": "gebco_latest_2",
    "query_layers": "gebco_latest_2",
    "BBOX": "33,138,38,143",
    "info_format": "text/plain",
    "x": "400",
    "y": "400",
    "width": "951",
    "height": "400",
    "version": "1.3.0"
}

p = re.compile(r"value_list\s*=\s*\'(-?\d*\.?\d*)")

def get_elevations(m, payload, coords):
    payload["width"] = int(m.right-m.left)
    payload["height"] = int(m.bottom-m.top)
    payload["BBOX"] = f"{m.south:.5f},{m.west:.5f},{m.north:.5f},{m.east:.5f}"
    elevations = []
    for long, lat in coords:
        payload["x"] = int((long-m.west)/(m.east-m.west)*payload["width"])
        payload["y"] = int((lat-m.north)/(m.south-m.north)*payload["height"])
        #print(payload["width"], payload["height"])
        #print(payload["x"], payload["y"])
        #print(payload["BBOX"])
        gebcoStr = ""
        url = 'https://www.gebco.net/data_and_products/gebco_web_services/web_map_service/mapserv'
        req = requests.get(url, params=payload)
        # https://requests.kennethreitz.org/en/latest/user/advanced/#prepared-requests
        #req = requests.Request('GET', url, data=payload)
        #prepped = req.prepare()
        #print(prepped.body)
        gebcoStr = req.text
        success = False
        if gebcoStr:
            mm = p.search(gebcoStr)
            if mm:
                elevations.append(int(mm.groups(0)[0]))
                success = True
        if not success:
            elevations.append(None)
    return elevations

In [5]:
get_elev = True

In [6]:
draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#00FFFF",
        "weight": 6,
        "opacity": 1.0,
        "fillOpacity": 0
    }
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    #feature_collection['features'].append(geo_json)
    print(action)
    fname = "./data/geojson_from_map.json"
    if str(action)=="deleted": return None
    with open(fname, 'w') as fh:
        coords = geo_json["geometry"]["coordinates"]
        KP = calc_line_KPs(coords)
        geo_json["properties"]["style"]["fillOpacity"] = 0
        geo_json["properties"]["KP"] = KP
        if get_elev: geo_json["properties"]["elevation"] = get_elevations(m, payload, coords)
        json.dump(geo_json, fh)
        print(f"geojson data written to file {fname}")
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

In [7]:
geojson_file1 = "./data/Bluestream.json"
with open(geojson_file1, 'r') as f:
    east_data = json.load(f)
east_json = GeoJSON(name='deepwater route', data=east_data)
m.add_layer(east_json)

In [8]:
m

Map(center=[42.9, 36.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [9]:
#dir(m)
pass